In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.core import value
import opf_utils
import os

# Load the data from the Excel file
data = pd.ExcelFile('OPFData.xlsx')
node_data = data.parse('NodeData')
real_admittance_matrix = data.parse('RealAdmittanceMatrix', header=None)
imaginary_admittance_matrix = data.parse('ImaginaryAdmittanceMatrix', header=None)

# Check dimensions
num_nodes = len(node_data)



assert real_admittance_matrix.shape == (num_nodes, num_nodes), "Real admittance matrix dimensions do not match the number of nodes"
assert imaginary_admittance_matrix.shape == (num_nodes, num_nodes), "Imaginary admittance matrix dimensions do not match the number of nodes"

# Define the Pyomo model
model = pyo.ConcreteModel()

# Generate a list of nodes based on the index
nodes = list(range(1, num_nodes + 1))

# Define the sets
model.N = pyo.Set(initialize=nodes)
model.G = pyo.Set(initialize=[i for i in nodes if node_data.iloc[i-1]['PGmax'] > 0])
model.L = pyo.Set(initialize=[i for i in nodes if node_data.iloc[i-1]['Pload'] > 0])

# Create dictionaries for parameter initialization
Pd_dict = {i+1: node_data['Pload'].iloc[i] for i in range(num_nodes)}
Qd_dict = {i+1: node_data['Qload'].iloc[i] for i in range(num_nodes)}
Pmin_dict = {i+1: node_data['PGmin'].iloc[i] for i in range(num_nodes) if node_data['PGmax'].iloc[i] > 0}
Pmax_dict = {i+1: node_data['PGmax'].iloc[i] for i in range(num_nodes) if node_data['PGmax'].iloc[i] > 0}
Qmin_dict = {i+1: node_data['QGmin'].iloc[i] for i in range(num_nodes) if node_data['PGmax'].iloc[i] > 0}
Qmax_dict = {i+1: node_data['QGmax'].iloc[i] for i in range(num_nodes) if node_data['PGmax'].iloc[i] > 0}
a_dict = {i+1: node_data['a'].iloc[i] for i in range(num_nodes) if node_data['PGmax'].iloc[i] > 0}
b_dict = {i+1: node_data['b'].iloc[i] for i in range(num_nodes) if node_data['PGmax'].iloc[i] > 0}
c_dict = {i+1: node_data['c'].iloc[i] for i in range(num_nodes) if node_data['PGmax'].iloc[i] > 0}
Vmax_dict = {i+1: node_data['Vmax'].iloc[i] for i in range(num_nodes)}
Vmin_dict = {i+1: node_data['Vmin'].iloc[i] for i in range(num_nodes)}
VAnglemax_dict = {i+1: node_data['VAnglemax'].iloc[i] for i in range(num_nodes)}
VAnglemin_dict = {i+1: node_data['VAnglemin'].iloc[i] for i in range(num_nodes)}
# Define the parameters
model.Pd = pyo.Param(model.N, initialize=Pd_dict)
model.Qd = pyo.Param(model.N, initialize=Qd_dict)
model.Pmin = pyo.Param(model.G, initialize=Pmin_dict)
model.Pmax = pyo.Param(model.G, initialize=Pmax_dict)
model.Qmin = pyo.Param(model.G, initialize=Qmin_dict)
model.Qmax = pyo.Param(model.G, initialize=Qmax_dict)
model.a = pyo.Param(model.G, initialize=a_dict)
model.b = pyo.Param(model.G, initialize=b_dict)
model.c = pyo.Param(model.G, initialize=c_dict)
model.Vmax = pyo.Param(model.N, initialize=Vmax_dict)
model.Vmin = pyo.Param(model.N, initialize=Vmin_dict)
model.VAnglemax = pyo.Param(model.N, initialize=VAnglemax_dict)
model.VAnglemin = pyo.Param(model.N, initialize=VAnglemin_dict)
# Define the admittance matrix
model.Gik = pyo.Param(model.N, model.N, initialize={(i+1, j+1): real_admittance_matrix.iloc[i, j] for i in range(num_nodes) for j in range(num_nodes)})
model.Bik = pyo.Param(model.N, model.N, initialize={(i+1, j+1): imaginary_admittance_matrix.iloc[i, j] for i in range(num_nodes) for j in range(num_nodes)})

# Define the variables
model.PG = pyo.Var(model.G, within=pyo.NonNegativeReals)
model.QG = pyo.Var(model.G, within=pyo.Reals)
model.V = pyo.Var(model.N, within=pyo.NonNegativeReals, initialize=1)
model.delta = pyo.Var(model.N, within=pyo.Reals, initialize=0)

# Define the objective function
def objective_rule(model):
    return sum(model.a[g] * model.PG[g]**2 + model.b[g] * model.PG[g] + model.c[g] for g in model.G)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Real power balance constraint
def real_power_balance_rule(model, i):
    if i in model.G:
        return model.PG[i] - model.Pd[i] == sum(
            model.V[i] * model.V[j] * (
                model.Gik[i, j] * pyo.cos(model.delta[i] - model.delta[j]) +
                model.Bik[i, j] * pyo.sin(model.delta[i] - model.delta[j])
            ) for j in model.N if (i, j) in model.Gik
        )
    else:
        return - model.Pd[i] == sum(
            model.V[i] * model.V[j] * (
                model.Gik[i, j] * pyo.cos(model.delta[i] - model.delta[j]) +
                model.Bik[i, j] * pyo.sin(model.delta[i] - model.delta[j])
            ) for j in model.N if (i, j) in model.Gik
        )
model.real_power_balance = pyo.Constraint(model.N, rule=real_power_balance_rule)

# Reactive power balance constraint
def reactive_power_balance_rule(model, i):
    if i in model.G:
        return model.QG[i] - model.Qd[i] == sum(
            model.V[i] * model.V[j] * (
                model.Gik[i, j] * pyo.sin(model.delta[i] - model.delta[j]) -
                model.Bik[i, j] * pyo.cos(model.delta[i] - model.delta[j])
            ) for j in model.N if (i, j) in model.Gik
        )
    else:
        return - model.Qd[i] == sum(
            model.V[i] * model.V[j] * (
                model.Gik[i, j] * pyo.sin(model.delta[i] - model.delta[j]) -
                model.Bik[i, j] * pyo.cos(model.delta[i] - model.delta[j])
            ) for j in model.N if (i, j) in model.Gik
        )
model.reactive_power_balance = pyo.Constraint(model.N, rule=reactive_power_balance_rule)

# Generation limits
model.PG_min = pyo.Constraint(model.G, rule=lambda model, g: model.PG[g] >= model.Pmin[g])
model.PG_max = pyo.Constraint(model.G, rule=lambda model, g: model.PG[g] <= model.Pmax[g])
model.QG_min = pyo.Constraint(model.G, rule=lambda model, g: model.QG[g] >= model.Qmin[g])
model.QG_max = pyo.Constraint(model.G, rule=lambda model, g: model.QG[g] <= model.Qmax[g])

# Voltage limits
model.V_min = pyo.Constraint(model.N, rule=lambda model, i: model.V[i] >= model.Vmin[i])
model.V_max = pyo.Constraint(model.N, rule=lambda model, i: model.V[i] <= model.Vmax[i])

# Voltage angle limits
model.VAngle_min = pyo.Constraint(model.N, rule=lambda model, i: model.delta[i] >= model.VAnglemin[i])
model.VAngle_max = pyo.Constraint(model.N, rule=lambda model, i: model.delta[i] <= model.VAnglemax[i])


os.environ['NEOS_EMAIL'] = 'jihad.jundi@tum.de'  # Use your valid email address

# Solve the model
solver_manager = pyo.SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='ipopt', tee=True)

# Check if solver found an optimal solution
if (results.solver.status != pyo.SolverStatus.ok) or (results.solver.termination_condition != pyo.TerminationCondition.optimal):
    raise RuntimeError("Solver failed to find an optimal solution")

# Extract results
results_dict = {
    'name': 'Your Team Name',
    'task': 'AC',
    'cost': value(model.obj),
    'nodes': [
        {
            'id': i,
            'voltage_angle': value(model.delta[i]),
            'voltage_magnitude': value(model.V[i]),
            'power': value(model.PG[i]) if i in model.G else 0,
            'reactive_power': value(model.QG[i]) if i in model.G else 0
        }
        for i in model.N
    ]
}


           


In [3]:
def save_results(results, filename):
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)

# Save results to a JSON file
save_results(results_dict, 'Results_4_AC_Group_YourGroup.json')

NameError: name 'json' is not defined

In [5]:
print(results_dict)

{'name': 'Your Team Name', 'task': 'AC', 'cost': 0.4032767056598064, 'nodes': [{'id': 1, 'voltage_angle': 5.362121404535845e-18, 'voltage_magnitude': 0.9999999996396735, 'power': 0.9451004934581662, 'reactive_power': 0.6099465532593968}, {'id': 2, 'voltage_angle': -0.14796050644340608, 'voltage_magnitude': 0.9839069134742487, 'power': 0, 'reactive_power': 0}, {'id': 3, 'voltage_angle': -0.06511622960950143, 'voltage_magnitude': 0.9652310334859262, 'power': 0.19409266615403353, 'reactive_power': -0.08980128253965367}, {'id': 4, 'voltage_angle': -0.16220787127637146, 'voltage_magnitude': 0.971064697193147, 'power': 0.05666290666882956, 'reactive_power': 0.19999779928522157}, {'id': 5, 'voltage_angle': -0.15284187862071394, 'voltage_magnitude': 0.9500002663423008, 'power': 0, 'reactive_power': 0}]}


In [46]:
print(model.PG)

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
